In [9]:
import backtrader as bt
import pandas as pd
import datetime
import logging
import os

# Configuração básica de log
logging.basicConfig(format='%(asctime)s - %(message)s', level=logging.INFO)

In [10]:
class AurumData(bt.feeds.PandasData):
    """
    Mapeia as colunas do Pandas para o Backtrader.
    Adicionamos 'sentiment' e 'news_vol'.
    """
    lines = ('aurum_score', 'roe', 'roic', 'volatility', 'sentiment', 'news_vol',)

    params = (
        ('datetime', None),
        ('open', 'Adj Close'),  
        ('high', 'Adj Close'),
        ('low', 'Adj Close'),
        ('close', 'Adj Close'), 
        ('volume', -1), 
        ('openinterest', None),
        
        ('aurum_score', 'aurum_quality_score'),
        ('roe', 'ROE'),
        ('roic', 'ROIC'),
        ('volatility', 'VOLATILIDADE'), 
        ('sentiment', 'SENTIMENT_SCORE'),
        ('news_vol', 'NEWS_VOLUME'),    
    )  

In [ ]:
class AurumRankingStrategy(bt.Strategy):
    params = (
        ('top_n', 5),         
        ('rebalance_days', 1), 
        ('reserve_cash', 0.05), 
        ('sentiment_filter', True), 
        ('min_sentiment', -0.3),    
    )

    def log(self, txt, dt=None):
        dt = dt or self.datas[0].datetime.date(0)
        print(f'{dt.isoformat()} | {txt}')

    def __init__(self):
        self.inds = {}
        for d in self.datas:
            self.inds[d] = {
                'score': d.aurum_score,
                'sentiment': d.sentiment,
                'name': d._name
            }
        self.timer_count = 0

    def next(self):
        self.timer_count += 1
        if self.timer_count < self.params.rebalance_days:
            return
        
        self.timer_count = 0
        self.log(f'--- REBALANCEAMENTO (Cash: {self.broker.get_cash():.2f}) ---')

        candidates = []
        for d in self.datas:
            if len(d) > 0 and d.close[0] > 0 and d.aurum_score[0] > 0:
                candidates.append((d, d.aurum_score[0], d.sentiment[0]))

        candidates.sort(key=lambda x: x[1], reverse=True)

        buy_list = []
        
        for d, score, sent in candidates:
            if len(buy_list) >= self.params.top_n:
                break
                
            if self.params.sentiment_filter:
                if sent < self.params.min_sentiment:
                    self.log(f"🚫 VETO: {d._name} (Score: {score:.1f}, Sentimento Ruim: {sent:.2f})")
                    continue
            
            buy_list.append(d)
        
        buy_names = [d._name for d in buy_list]
        self.log(f'TOP {self.params.top_n} ESCOLHIDOS: {buy_names}')

        for d in self.datas:
            if self.getposition(d).size > 0:
                if d not in buy_list:
                    self.log(f'VENDENDO {d._name} (Saiu do Portfolio)')
                    self.close(d) 

        if len(buy_list) > 0:
            target_pct = (1.0 - self.params.reserve_cash) / len(buy_list)
            for d in buy_list:
                self.order_target_percent(d, target=target_pct)



In [ ]:
def filtrar_tickers_problematicos(df, limite_alta=1.0, limite_baixa=-0.8):
    print("\n🧹 Iniciando Filtro de Sanidade...")
    tickers_originais = df['ticker'].unique()
    df['retorno'] = df.groupby('ticker')['Adj Close'].pct_change()
    bad_tickers = df[(df['retorno'] > limite_alta) | (df['retorno'] < limite_baixa)]['ticker'].unique()
    
    if len(bad_tickers) > 0:
        print(f"🚫 BANINDO {len(bad_tickers)} tickers problemáticos: {list(bad_tickers)}")
        df_limpo = df[~df['ticker'].isin(bad_tickers)].copy()
        print(f"✅ Tickers restantes: {len(df_limpo['ticker'].unique())}")
        return df_limpo
    return df

In [12]:
def run_strategy():
    print("--- 🦁 INICIANDO BACKTEST AURUM (V4 - COM SENTIMENTO) ---")
    cerebro = bt.Cerebro()

    path_data = "../data/aurum_master_features_final.parquet"
    
    if not os.path.exists(path_data):
        print(f"❌ Erro: Arquivo {path_data} não encontrado. Rode o Merge V6 primeiro.")
        return

    try:
        df_master = pd.read_parquet(path_data)
        df_master['date'] = pd.to_datetime(df_master['date'])
        df_master = df_master.sort_values('date')
        
        if 'SENTIMENT_SCORE' not in df_master.columns:
            df_master['SENTIMENT_SCORE'] = 0.0
        if 'NEWS_VOLUME' not in df_master.columns:
            df_master['NEWS_VOLUME'] = 0
            
    except Exception as e:
        print(f"Erro leitura: {e}"); return

    df_master = filtrar_tickers_problematicos(df_master)

    start_date = '2023-01-01'
    end_date = '2025-12-31'
    mask = (df_master['date'] >= start_date) & (df_master['date'] <= end_date)
    df_filtered = df_master.loc[mask]
    
    tickers = df_filtered['ticker'].unique()
    print(f"\nCarregando {len(tickers)} feeds...")

    for ticker in tickers:
        df_ticker = df_filtered[df_filtered['ticker'] == ticker].copy()
        df_ticker = df_ticker.set_index('date').sort_index()

        if len(df_ticker) < 6: continue 

        data_feed = AurumData(
            dataname=df_ticker, name=ticker,
            fromdate=pd.to_datetime(start_date), 
            todate=pd.to_datetime(end_date)
        )
        cerebro.adddata(data_feed)

    cerebro.addstrategy(AurumRankingStrategy, top_n=5, sentiment_filter=True)
    cerebro.broker.setcash(100000.0)
    cerebro.broker.setcommission(commission=0.0005) # 0.05%
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')

    print(f'\n💰 Saldo Inicial: R$ {cerebro.broker.getvalue():,.2f}')
    results = cerebro.run()
    strat = results[0]
    
    final_val = cerebro.broker.getvalue()
    roi = ((final_val - 100000) / 100000) * 100
    
    print(f'💰 Saldo Final:   R$ {final_val:,.2f}')
    print(f'📈 Retorno Total: {roi:.2f}%')
    try:
        print(f'📊 Sharpe Ratio:  {strat.analyzers.sharpe.get_analysis()["sharperatio"]:.3f}')
        print(f'📉 Max Drawdown:  {strat.analyzers.drawdown.get_analysis()["max"]["drawdown"]:.2f}%')
    except: pass

if __name__ == '__main__':
    run_strategy()

--- 🦁 INICIANDO BACKTEST AURUM (V4 - COM SENTIMENTO) ---

🧹 Iniciando Filtro de Sanidade...
🚫 BANINDO 19 tickers problemáticos: ['COGN3.SA', 'PRIO3.SA', 'GOAU4.SA', 'USIM5.SA', 'FLRY3.SA', 'PCAR3.SA', 'MGLU3.SA', 'AZZA3.SA', 'SLCE3.SA', 'PSSA3.SA', 'CPLE3.SA', 'IRBR3.SA', 'LREN3.SA', 'CSAN3.SA', 'CPLE5.SA', 'UGPA3.SA', 'TOTS3.SA', 'CEAB3.SA', 'TEND3.SA']
✅ Tickers restantes: 75

Carregando 75 feeds...

💰 Saldo Inicial: R$ 100,000.00
2025-06-30 | --- REBALANCEAMENTO (Cash: 100000.00) ---
2025-06-30 | TOP 5 ESCOLHIDOS: ['CURY3.SA', 'IGTI11.SA', 'VIVA3.SA', 'ALOS3.SA', 'VALE3.SA']
2025-07-01 | --- REBALANCEAMENTO (Cash: 5024.94) ---
2025-07-01 | TOP 5 ESCOLHIDOS: ['CURY3.SA', 'IGTI11.SA', 'VIVA3.SA', 'ALOS3.SA', 'VALE3.SA']
2025-07-31 | --- REBALANCEAMENTO (Cash: 5024.94) ---
2025-07-31 | TOP 5 ESCOLHIDOS: ['CURY3.SA', 'IGTI11.SA', 'VIVA3.SA', 'ALOS3.SA', 'VALE3.SA']
2025-08-01 | --- REBALANCEAMENTO (Cash: 4807.88) ---
2025-08-01 | TOP 5 ESCOLHIDOS: ['CURY3.SA', 'IGTI11.SA', 'VIVA3.SA', '